# Xây dựng Database TEST (Queries) (v2 - inner_content) từ các Span Adaptation Vectors của bộ Test (GramVar và ParaVE).

## Mục đích

- Gộp toàn bộ span adaptation vectors (Inner Content) của bộ dữ liệu TEST (GramVar và ParaVE) thành:
  - Một tensor duy nhất chứa vector của tất cả span trong tập test.
  - Một file metadata chứa thông tin mô tả chi tiết cho từng span.
- Database TEST này sẽ được dùng làm:
  - Tập truy vấn (queries) để đánh giá, thực nghiệm retrieval hoặc các mô hình downstream khác so với database train.

## Đầu vào

1. Đường dẫn cơ sở
   - `drive_base_path`:
     - `/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep`.

2. Các thư mục TEST chứa span vectors (Inner Content)
   - Danh sách thư mục test chính: `test_v2_dirs`:
     - `span_adaptation_vectors_test_gramvar_inner_content`
     - `span_adaptation_vectors_test_parave_inner_content`
   - Bên trong mỗi thư mục test chính:
     - Có nhiều thư mục con, mỗi thư mục ứng với một corpus hoặc một động từ:
       - Ví dụ:
         - `span_adaptation_vectors_test_gramvar_inner_content/run/`
         - `span_adaptation_vectors_test_parave_inner_content/eat/`.

3. Các file `.pt` bên trong mỗi thư mục corpus
   - Mỗi file đại diện cho một span vector:
     - Ví dụ:
       - `sentence_12_ARG_0.pt`
       - `sentence_45_ARG_MANNER.pt`
   - Được trích thông tin bằng regex:
     - `file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")`
     - Group 1: `sentence_idx` (số câu).
     - Group 2: `span_id` (ví dụ: `ARG_0`, `ARG_MANNER`).

4. Cấu trúc dữ liệu bên trong file `.pt`
   - Mỗi `.pt` là một dictionary:
     - Các key layer mong đợi:
       - `"layer_1", "layer_2", ..., "layer_12"`.
     - Mỗi value là một vector span ở layer tương ứng, cùng kích thước D.
   - Script kiểm tra đầy đủ 12 layer thông qua:
     - `EXPECTED_LAYER_KEYS = [f"layer_{i}" for i in range(1, 13)]`.

5. Thư mục output cho database TEST
   - `output_db_dir`:
     - `cached_databases` (tạo nếu chưa tồn tại).
   - Hai file output:
     - Vectors: `combined_TEST_db_vectors_v2_inner_content.pt`
     - Metadata: `combined_TEST_db_metadata_v2_inner_content.json`.

## Đầu ra

1. Tensor vector TEST đã gộp
   - File:
     - `cached_databases/combined_TEST_db_vectors_v2_inner_content.pt`.
   - Nội dung:
     - Một tensor duy nhất:
       - Kích thước: `[N, 12, D]`
         - N: tổng số span hợp lệ trong toàn bộ tập TEST.
         - 12: số lượng layer (layer_1 đến layer_12).
         - D: kích thước vector span trên mỗi layer.
     - Được tạo bằng:
       - `combined_vectors = torch.stack(all_db_vectors, dim=0)`.

2. File metadata TEST tương ứng
   - File:
     - `cached_databases/combined_TEST_db_metadata_v2_inner_content.json`.
   - Nội dung:
     - Một list JSON, mỗi phần tử là một span:
       - `corpus_id`: tên thư mục corpus (ví dụ: tên động từ).
       - `sentence_idx`: chỉ số câu, lấy từ tên file.
       - `span_id`: mã span, ví dụ: `ARG_0`, `ARG_MANNER`.
       - `source_dir`: thư mục test gốc (ví dụ: `span_adaptation_vectors_test_gramvar_inner_content`).
       - `vector_file_path`: đường dẫn đầy đủ tới file `.pt` gốc.

   - Ví dụ:
     ```json
     {
         "corpus_id": "run",
         "sentence_idx": 12,
         "span_id": "ARG_0",
         "source_dir": "span_adaptation_vectors_test_gramvar_inner_content",
         "vector_file_path": "/.../span_adaptation_vectors_test_gramvar_inner_content/run/sentence_12_ARG_0.pt"
     }
     ```

## Quy trình

### 1. Thiết lập đường dẫn và cấu hình

- In thông báo bắt đầu:
  - Bước 1: Xây dựng Database TEST (Queries) (v2 - inner_content).
- Thiết lập:
  - `drive_base_path`.
  - `test_v2_dirs`: hai thư mục test chính (GramVar và ParaVE).
- Thiết lập thư mục output:
  - `output_db_dir = cached_databases` (tạo nếu chưa tồn tại).
- Thiết lập tên hai file output:
  - `output_vectors_file = combined_TEST_db_vectors_v2_inner_content.pt`.
  - `output_metadata_file = combined_TEST_db_metadata_v2_inner_content.json`.

### 2. Khởi tạo regex và biến hỗ trợ

- Regex parse tên file:
  - `file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")`.
- Danh sách key layer bắt buộc:
  - `EXPECTED_LAYER_KEYS = ["layer_1", ..., "layer_12"]`.
- Hai list dùng để lưu:
  - `all_db_vectors`: danh sách tất cả vector span (tensor [12, D]).
  - `all_db_metadata`: danh sách metadata tương ứng.

### 3. Hàm `process_pt_file(file_path, corpus_id, main_dir_name)`

- Mục đích:
  - Xử lý một file `.pt` đơn lẻ và trích xuất:
    - Vector span `[12, D]`.
    - Metadata tương ứng.

- Bước xử lý:

1. Lấy tên file:
   - `filename = os.path.basename(file_path)`.
2. Áp dụng regex:
   - `match = file_pattern.match(filename)`.
   - Nếu không khớp:
     - In cảnh báo và trả về `(None, None)`.

3. Lấy thông tin từ regex:
   - `sentence_idx = int(match.group(1))`.
   - `span_id = match.group(2)` (ví dụ `ARG_0`, `ARG_MANNER`).

4. Load dữ liệu `.pt`:
   - `span_vector_data = torch.load(file_path, map_location='cpu')`.
   - Nếu không phải dict:
     - Bỏ qua.

5. Kiểm tra đủ 12 layer:
   - `if not all(key in span_vector_data for key in EXPECTED_LAYER_KEYS):`
     - Nếu thiếu layer, bỏ qua file.

6. Gộp vector các layer:
   - `all_layers_vec = torch.stack([span_vector_data[key] for key in EXPECTED_LAYER_KEYS], dim=0)`.
   - Kích thước: `[12, D]`.

7. Tạo metadata:
   - `metadata = {`
     - `'corpus_id': corpus_id,`
     - `'sentence_idx': sentence_idx,`
     - `'span_id': span_id,`
     - `'source_dir': main_dir_name,`
     - `'vector_file_path': file_path`
     - `}`.

8. Trả về:
   - `(all_layers_vec, metadata)`.

- Nếu có exception:
  - In lỗi và trả về `(None, None)`.

### 4. Quét các thư mục test và xây dựng database

- In số lượng thư mục test chính:
  - Ví dụ: 2 (GramVar + ParaVE).
- Với mỗi `main_dir_path` trong `test_v2_dirs`:
  - `main_dir_name = os.path.basename(main_dir_path)`.
  - In thông báo đang quét thư mục test chính đó.

1. Tìm tất cả thư mục corpus bên trong:
   - `corpus_dirs = [d for d in glob.glob(os.path.join(main_dir_path, '*')) if os.path.isdir(d)]`.
   - Mỗi `corpus_dir` tương ứng với một corpus/động từ.

2. Nếu không có corpus:
   - In lỗi và `continue`.

3. Khởi tạo `tqdm`:
   - `pbar = tqdm(total=len(corpus_dirs), desc=f"Corpus ({main_dir_name})", unit="corpus")`.

4. Với mỗi `corpus_dir`:
   - `corpus_id = os.path.basename(corpus_dir)`.
   - Lấy tất cả file `.pt`:
     - `pt_files = glob.glob(os.path.join(corpus_dir, '*.pt'))`.
   - Nếu không có `.pt`:
     - `pbar.update(1)` và tiếp tục corpus tiếp theo.

   - Khởi tạo `count_this_corpus = 0`.
   - Với mỗi `file_path` trong `pt_files`:
     - Gọi `process_pt_file(file_path, corpus_id, main_dir_name)`.
     - Nếu vector và metadata hợp lệ:
       - Append vào `all_db_vectors` và `all_db_metadata`.
       - Tăng `count_this_corpus`.

   - In log ngắn:
     - Số span đã xử lý cho corpus đó.
   - `pbar.update(1)` sau khi xử lý xong corpus.

5. Đóng `pbar` sau khi hoàn thành thư mục test hiện tại.

### 5. Gộp tensor và lưu file vector TEST

- Sau khi quét xong:
  - In tổng số vector hợp lệ trong tập test: `len(all_db_vectors)`.

- Nếu `all_db_vectors` rỗng:
  - In cảnh báo: không tìm thấy vector nào (TEST).
- Nếu có vector:
  - Gộp:
    - `combined_vectors = torch.stack(all_db_vectors, dim=0)`.
  - In kích thước:
    - `combined_vectors.shape` (dự kiến `[N, 12, D]`).
  - Lưu vector:
    - `torch.save(combined_vectors, output_vectors_file)`.

- Nếu lỗi khi lưu vector:
  - In lỗi nhưng không `exit()`, để vẫn lưu metadata.

### 6. Lưu file metadata TEST

- Dù có hay không có vector (trừ khi gặp lỗi ghi file):
  - Ghi file JSON:
    - `json.dump(all_db_metadata, f, indent=4)`.
- In thông báo đã lưu file metadata, và thông báo hoàn tất bước 1 (DB TEST).

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
import re # Thêm regex

print("--- Bắt đầu Bước 1: Xây dựng Database TEST (Queries) (v2 - inner_content) ---")

# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# *** CẤU HÌNH QUÉT (TEST) ***
# Thư mục INPUT: Trỏ đến các thư mục TEST
test_v2_dirs = [
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_gramvar_inner_content'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_parave_inner_content')
]

# Thư mục OUTPUT: Nơi lưu database đã gộp
output_db_dir = os.path.join(drive_base_path, 'cached_databases')
os.makedirs(output_db_dir, exist_ok=True)

# Tên file output cho database TEST
output_vectors_file = os.path.join(output_db_dir, 'combined_TEST_db_vectors_v2_inner_content.pt')
output_metadata_file = os.path.join(output_db_dir, 'combined_TEST_db_metadata_v2_inner_content.json')

print(f"Các thư mục input (TEST): {test_v2_dirs}")
print(f"File vector output (TEST): {output_vectors_file}")
print(f"File metadata output (TEST): {output_metadata_file}")

# --- 2. CÁC BIẾN VÀ HÀM HỖ TRỢ ---
# (Phần này giống hệt script trước)

file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")
EXPECTED_LAYER_KEYS = [f"layer_{i}" for i in range(1, 13)] # layer_1 đến layer_12

all_db_vectors = []
all_db_metadata = []

def process_pt_file(file_path, corpus_id, main_dir_name):
    """
    Xử lý một file .pt, trích xuất vector và metadata.
    """
    filename = os.path.basename(file_path)
    match = file_pattern.match(filename)

    if not match:
        print(f"Cảnh báo: Bỏ qua file (không khớp pattern) {file_path}")
        return None, None

    try:
        sentence_idx = int(match.group(1))
        span_id = match.group(2) # Đây là ARG_...

        span_vector_data = torch.load(file_path, map_location='cpu')

        if not isinstance(span_vector_data, dict):
            print(f"Cảnh báo: Bỏ qua file (không phải dict) {file_path}")
            return None, None

        if not all(key in span_vector_data for key in EXPECTED_LAYER_KEYS):
            print(f"Cảnh báo: Bỏ qua file (không chứa đủ 12 lớp) {file_path}")
            return None, None

        all_layers_vec = torch.stack(
            [span_vector_data[key] for key in EXPECTED_LAYER_KEYS],
            dim=0
        )

        metadata = {
            'corpus_id': corpus_id,
            'sentence_idx': sentence_idx,
            'span_id': span_id,
            'source_dir': main_dir_name,
            'vector_file_path': file_path
        }
        return all_layers_vec, metadata
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None, None

# --- 3. QUÁ TRÌNH QUÉT VÀ XÂY DỰNG DATABASE (TEST) ---

print(f"Tìm thấy {len(test_v2_dirs)} thư mục test chính. Bắt đầu quét...")

for main_dir_path in test_v2_dirs:
    main_dir_name = os.path.basename(main_dir_path)
    print(f"\n--- Đang quét thư mục chính (TEST): {main_dir_name} ---")

    corpus_dirs = [d for d in glob.glob(os.path.join(main_dir_path, '*')) if os.path.isdir(d)]

    if not corpus_dirs:
        print(f"LỖI: Không tìm thấy thư mục corpus nào trong {main_dir_path}. Bỏ qua.")
        continue

    print(f"Tìm thấy {len(corpus_dirs)} thư mục corpus trong {main_dir_name}. Bắt đầu xử lý...")

    pbar = tqdm(total=len(corpus_dirs), desc=f"Corpus ({main_dir_name})", unit="corpus")

    for corpus_dir in corpus_dirs:
        corpus_id = os.path.basename(corpus_dir)
        pt_files = glob.glob(os.path.join(corpus_dir, '*.pt'))

        if not pt_files:
            pbar.update(1)
            continue

        count_this_corpus = 0
        for file_path in pt_files:
            vector, metadata = process_pt_file(file_path, corpus_id, main_dir_name)

            if vector is not None and metadata is not None:
                all_db_vectors.append(vector)
                all_db_metadata.append(metadata)
                count_this_corpus += 1

        pbar.write(f"Đang xử lý Corpus: {corpus_id} (Từ {main_dir_name}) -> Đã xử lý {count_this_corpus} span.")
        pbar.update(1)

    pbar.close()

print("\n--- QUÁ TRÌNH QUÉT (TEST) ĐÃ HOÀN TẤT! ---")
print(f"Tổng cộng tìm thấy {len(all_db_vectors)} vector (TEST) hợp lệ.")

if not all_db_vectors:
    print("KHÔNG TÌM THẤY VECTOR NÀO (TEST). Sẽ tạo file metadata rỗng.")
else:
    # --- Lưu file Vector (TEST) ---
    try:
        print("Đang gộp các tensor (TEST) thành 1 file .pt...")
        combined_vectors = torch.stack(all_db_vectors, dim=0)
        print(f" -> Kích thước tensor gộp (TEST): {combined_vectors.shape}")
        torch.save(combined_vectors, output_vectors_file)
        print(f" -> Đã lưu thành công file vector (TEST): {output_vectors_file}")
    except Exception as e:
        print(f"LỖI khi lưu file vector (TEST): {e}")
        # Bỏ exit() để vẫn lưu metadata
        # exit()

# --- Lưu file Metadata (TEST) ---
try:
    print("Đang lưu file metadata (TEST) .json...")
    with open(output_metadata_file, 'w', encoding='utf-8') as f:
        json.dump(all_db_metadata, f, indent=4)
    print(f" -> Đã lưu thành công file metadata (TEST): {output_metadata_file}")
except Exception as e:
    print(f"LỖI NGHIÊM TRỌNG khi lưu file metadata (TEST): {e}")

print("\n--- BƯỚC 1 (XÂY DỰNG DB TEST) HOÀN TẤT! ---")

--- Bắt đầu Bước 1: Xây dựng Database TEST (Queries) (v2 - inner_content) ---
Các thư mục input (TEST): ['/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_gramvar_inner_content', '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_parave_inner_content']
File vector output (TEST): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_TEST_db_vectors_v2_inner_content.pt
File metadata output (TEST): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_TEST_db_metadata_v2_inner_content.json
Tìm thấy 2 thư mục test chính. Bắt đầu quét...

--- Đang quét thư mục chính (TEST): span_adaptation_vectors_test_gramvar_inner_content ---
Tìm thấy 35 thư mục corpus trong span_adaptation_vectors_test_gramvar_inner_content. Bắt đầu xử lý...


Corpus (span_adaptation_vectors_test_gramvar_inner_content):   0%|          | 0/35 [00:00<?, ?corpus/s]

Đang xử lý Corpus: begin_2 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 56 span.
Đang xử lý Corpus: begin_1 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 36 span.
Đang xử lý Corpus: translate_2 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 36 span.
Đang xử lý Corpus: transform_1 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 48 span.
Đang xử lý Corpus: delete (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 16 span.
Đang xử lý Corpus: confer (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 117 span.
Đang xử lý Corpus: splice_2 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 142 span.
Đang xử lý Corpus: translate_1 (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 88 span.
Đang xử lý Corpus: inhibit (Từ span_adaptation_vectors_test_gramvar_inner_content) -> Đã xử lý 42 span.
Đang xử lý Corpus: proliferate (Từ span_adaptation_

Corpus (span_adaptation_vectors_test_parave_inner_content):   0%|          | 0/35 [00:00<?, ?corpus/s]

Đang xử lý Corpus: catalyse (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 14 span.
Đang xử lý Corpus: result (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 62 span.
Đang xử lý Corpus: eliminate (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 11 span.
Đang xử lý Corpus: develop (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 2 span.
Đang xử lý Corpus: translate_3 (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 50 span.
Đang xử lý Corpus: transform_1 (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 6 span.
Đang xử lý Corpus: begin_2 (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 38 span.
Đang xử lý Corpus: block (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 25 span.
Đang xử lý Corpus: abolish (Từ span_adaptation_vectors_test_parave_inner_content) -> Đã xử lý 22 span.
Đang xử lý Corpus: translate_2 (Từ span_adaptation_vectors_test_par